In [ ]:
import os
import sys
import glob
import argparse
import matplotlib.pyplot as plt

from keras import __version__
from keras.applications.inception_v3 import InceptionV3, preprocess_input
from keras.models import Model
from keras.layers import Dense, Dropout, GlobalAveragePooling2D
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD


IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3
NB_EPOCHS = 10
BAT_SIZE = 32
FC_SIZE = 1024
DROP_RT = 0.5
NB_IV3_LAYERS_TO_FREEZE = 172


def get_nb_files(directory):
  """Get number of files by searching directory recursively"""
  if not os.path.exists(directory):
    return 0
  cnt = 0
  for r, dirs, files in os.walk(directory):
    for dr in dirs:
      cnt += len(glob.glob(os.path.join(r, dr + "/*")))
  return cnt


def setup_to_transfer_learn(model, base_model):
  """Freeze all layers and compile the model"""
  for layer in base_model.layers:
    layer.trainable = False
  model.compile(optimizer='rmsprop', loss='categorical_crossentropy', metrics=['accuracy'])


def add_new_last_layer(base_model, nb_classes):
  """Add last layer to the convnet

  Args:
    base_model: keras model excluding top
    nb_classes: # of classes

  Returns:
    new keras model with last layer
  """
  x = base_model.output
  x = GlobalAveragePooling2D()(x)
  x = Dense(FC_SIZE, activation='relu')(x)
  x = Dropout(DROP_RT)(x)
  predictions = Dense(nb_classes, activation='softmax')(x)
  model = Model(inputs=base_model.input, outputs=predictions)

  return model


def setup_to_finetune(model):
  """Freeze the bottom NB_IV3_LAYERS and retrain the remaining top layers.

  note: NB_IV3_LAYERS corresponds to the top 2 inception blocks in the inceptionv3 arch

  Args:
    model: keras model
  """
  for layer in model.layers[:NB_IV3_LAYERS_TO_FREEZE]:
     layer.trainable = False
  for layer in model.layers[NB_IV3_LAYERS_TO_FREEZE:]:
     layer.trainable = True
  model.compile(optimizer=SGD(lr=0.0001, momentum=0.9), loss='categorical_crossentropy', metrics=['accuracy'])


def train(train_dir, val_dir,
        nb_epoch=NB_EPOCHS, batch_size=BAT_SIZE,
        output_model_file='inceptionv3-ft.model',
        plot='store_true'):
  """Use transfer learning and fine-tuning to train a network on a new dataset"""
  nb_train_samples = get_nb_files(train_dir)
  nb_classes = len(glob.glob(train_dir + "/*"))
  nb_val_samples = get_nb_files(val_dir)
  nb_epoch = int(nb_epoch)
  batch_size = int(batch_size)

  # data prep
  train_datagen =  ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
  )
  test_datagen = ImageDataGenerator(
      preprocessing_function=preprocess_input,
      rotation_range=30,
      width_shift_range=0.2,
      height_shift_range=0.2,
      shear_range=0.2,
      zoom_range=0.2,
      horizontal_flip=True
  )

  train_generator = train_datagen.flow_from_directory(
    train_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
  )

  validation_generator = test_datagen.flow_from_directory(
    val_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=batch_size,
  )

  print(train_generator.class_indices)

  # setup model
  base_model = InceptionV3(weights='imagenet', include_top=False) #include_top=False excludes final FC layer
  model = add_new_last_layer(base_model, nb_classes)

  # transfer learning
  setup_to_transfer_learn(model, base_model)

  history_tl = model.fit_generator(
    train_generator,
    epochs=nb_epoch,
    steps_per_epoch=nb_train_samples/batch_size,
    validation_data=validation_generator,
    validation_steps=nb_val_samples/batch_size,
    class_weight='auto')

  # fine-tuning
  setup_to_finetune(model)

  history_ft = model.fit_generator(
    train_generator,
    steps_per_epoch=nb_train_samples/batch_size,
    epochs=nb_epoch,
    validation_data=validation_generator,
    validation_steps=nb_val_samples/batch_size,
    class_weight='auto')

  model.save(output_model_file)

  if plot:
    plot_training(history_ft)


def plot_training(history):
  acc = history.history['acc']
  val_acc = history.history['val_acc']
  loss = history.history['loss']
  val_loss = history.history['val_loss']
  epochs = range(len(acc))

  plt.plot(epochs, acc, 'r.')
  plt.plot(epochs, val_acc, 'r')
  plt.title('Training and validation accuracy')

  plt.figure()
  plt.plot(epochs, loss, 'r.')
  plt.plot(epochs, val_loss, 'r-')
  plt.title('Training and validation loss')
  plt.show()

In [ ]:
train(train_dir='/data/train', val_dir='/data/validation')

In [ ]:
import sys
import argparse
import numpy as np
from PIL import Image
import requests
from io import BytesIO
import matplotlib.pyplot as plt

from keras.preprocessing import image
from keras.models import load_model
from keras.applications.inception_v3 import preprocess_input
from keras.preprocessing.image import ImageDataGenerator

IM_WIDTH, IM_HEIGHT = 299, 299 #fixed size for InceptionV3

def predict_dir(model, predict_dir):
  """Run model prediction on image
  Args:
    model: keras model
    img: PIL format image
  Returns:
    list of predicted labels and their probabilities
  """
  # data prep
  predict_datagen =  ImageDataGenerator()

  predict_generator = predict_datagen.flow_from_directory(
    predict_dir,
    target_size=(IM_WIDTH, IM_HEIGHT),
    batch_size=256,
    class_mode=None,  # only data, no labels
    shuffle=False  # keep data in same order as labels
  )
    
  print(predict_generator.class_indices)
    
  num_files = len(predict_generator.filenames)
  print(predict_generator.filenames)
    
  preds = model.predict_generator(
    predict_generator,
    num_files,
    verbose=1)

  return preds

In [ ]:
test_dir = '/test'
model = 'inceptionv3-ft.model'

model = load_model(model)
preds = predict_dir(model, test_dir)
classes = preds.argmax(axis=-1)
print(preds)

In [16]:
import pandas as pd
import os

test_filenames = os.listdir('/test/unknown')

class_indices = {'nuts': 14, 'chocolate': 5, 'tomatosauce': 22, 'rice': 17, 'soda': 18, 'candy': 2, 'milk': 13, 'juice': 12, 'sugar': 20, 'cereal': 3, 'vinegar': 23, 'water': 24, 'spices': 19, 'pasta': 16, 'fish': 8, 'cake': 1, 'coffee': 6, 'chips': 4, 'honey': 10, 'beans': 0, 'corn': 7, 'flour': 9, 'tea': 21, 'jam': 11, 'oil': 15}
class_indices = dict(zip(class_indices.values(),class_indices.keys()))

classes = preds.argmax(axis=-1)
classes = [class_indices[x] for x in classes]

df = pd.DataFrame(list(zip(test_filenames, classes)))
df[0] = df[0].str.rstrip('.png')

df.to_csv('predictions.csv', sep=',', header=False, index=False)

print(df)

IOPub data rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_data_rate_limit`.


               0          1
0     test_1307c  chocolate
1     test_1234d        tea
2     test_1129a  chocolate
3     test_1056b     coffee
4     test_1145c     coffee
5     test_1218b        tea
6     test_1072d        tea
7     test_1315d     coffee
8     test_1137b        tea
9     test_1064c        tea
10    test_1048a        tea
11    test_1153d        tea
12    test_1226c        tea
13    test_1363a       rice
14    test_1290b     coffee
15    test_1282a  chocolate
16    test_1371b        tea
17    test_1092b     coffee
18    test_1165a        tea
19    test_1416b       rice
20    test_1270d     coffee
21    test_1343c  chocolate
22    test_1254b     coffee
23    test_1181c     coffee
24    test_1327a  chocolate
25    test_1432d     coffee
26    test_1246a     coffee
27    test_1173b     coffee
28    test_1424c        tea
29    test_1351d     coffee
...          ...        ...
1702  test_1242d     coffee
1703  test_1064b        tea
1704  test_1137a        tea
1705  test_1161d    